In [4]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from __future__ import print_function

In [14]:
def is_in_numpy_array(elem, array):
    """Checks whether elem is identical to any row in array."""
    return np.any(np.equal(elem, array).all(axis=1))

side_length = 100
num_lines = 10 # called L in project description

# All possible directions
directions_1d = np.array([1,0,0, -1,0,0]).reshape(-1,3)

directions_3d = [[0,0,1], [0,1,0], [1,0,0], [0,0,-1], [0,-1,0], [-1, 0, 0]]
directions_3d = np.array([np.array(d) for d in directions_3d])

# Detachement direction consists of all 3d movements that are not 1d-movements
directions_3d_detach = [d for d in directions_3d if not is_in_numpy_array(d, directions_1d)]
directions_3d_detach = np.array(directions_3d_detach)

def choose(array):
    """Choose a random element from an array."""
    # np.random.choice doesn't work with 2d-array :(
    idx = np.random.randint(0, len(array))
    return array[idx]

# Create lines parallel to x-axis
lines_y = np.random.choice(side_length, size=num_lines, replace=False)
lines_z = np.random.choice(side_length, size=num_lines, replace=False)

# Create targets along line
target_x = np.random.choice(side_length, size=num_lines, replace=False)
targets = np.vstack((target_x, lines_y, lines_z)).T

def number_of_sliding_steps(avg_sliding_len=10):
    """Returns number of sliding steps drawn from a neg. bin. distr.
    See also:
    https://stackoverflow.com/questions/40846992/alternative-parametrization-of-the-negative-binomial-in-scipy
    """
    v = float(avg_sliding_len + avg_sliding_len**2)
    p = 1.0 - (v - avg_sliding_len*1.0)/v
    return np.random.negative_binomial(1,p)

In [15]:
avg_sliding_len = 50
cost_ratio = 10 # how many times more expensive is 1d-diffusion per step?

position = np.random.randint(0, side_length, 3)
print("Initial position = {}".format(position))
iterations_3d = 0
iterations_1d = 0
attach_counter = 0

remaining_sliding_its = 0
is_sliding = False

while not is_in_numpy_array(position, targets):
    is_on_line = np.any((position[1] == lines_y) & (position[2] == lines_z))
    if is_on_line and not is_sliding:
        # Attach to line!
        remaining_sliding_its = number_of_sliding_steps(avg_sliding_len=avg_sliding_len)
        is_sliding = True
        attach_counter += 1
        print(str(iterations_3d + iterations_1d) + " Sliding for {}".format(remaining_sliding_its))
    if remaining_sliding_its > 0:
        assert is_on_line
        # Currently sliding, 1d brownian motion
        iterations_1d += 1
        remaining_sliding_its -= 1
        direction = choose(directions_1d)
    else:
        # Not sliding, 3d brownian motion
        iterations_3d += 1
        if is_sliding:
            # Just detached.
            assert remaining_sliding_its <= 0
            direction = choose(directions_3d_detach)
            is_sliding = False
            print("Detached!")
        else:            
            direction = choose(directions_3d)
        
    position += direction
    position = np.mod(position, np.array([side_length, side_length, side_length]))
    

total_cost = iterations_3d + cost_ratio * iterations_1d
    
iterations_3d, iterations_1d, total_cost, attach_counter

Initial position = [15 45 24]
9698 Sliding for 12
Detached!
9712 Sliding for 13
Detached!
13163 Sliding for 30


(13138, 27, 13408, 3)